In [1]:
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
import pandas as pd
import numpy as np
import pandas.rpy.common as com
import xgboost as xgb

from rpy2.robjects import pandas2ri
from gplearn.genetic import SymbolicTransformer
from sklearn.utils import check_random_state
import xgboost as xgb
import operator
from matplotlib import pylab as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
%matplotlib inline 

from sklearn.ensemble import ExtraTreesClassifier

from gplearn.genetic import SymbolicTransformer
from sklearn.utils import check_random_state
from sklearn.calibration import CalibratedClassifierCV

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:5: FutureWarning: The pandas.rpy module is deprecated and will be removed in a future version. We refer to external packages like rpy2. 
See here for a guide on how to port your code to rpy2: http://pandas.pydata.org/pandas-docs/stable/r_interface.html


In [ ]:
print('Load data...')
train = pd.read_csv("./input/train_poly_rmCorr.csv")
target = train['TARGET'].values
id_train= train['ID'].values
train = train.drop(['ID','TARGET'],axis=1)
test = pd.read_csv("./input/test_poly_rmCorr.csv")
id_test = test['ID'].values
test = test.drop(['ID'],axis=1)

#train['na_count'] = train.isnull().sum(axis=1)
#test['na_count'] = test.isnull().sum(axis=1)`

Load data...


In [ ]:
extc = ExtraTreesClassifier(n_estimators=1000,max_features= 50,criterion= 'entropy',min_samples_split= 4,
                            max_depth= 35, min_samples_leaf= 2, n_jobs = -1)
cal_clf = CalibratedClassifierCV(extc,method='isotonic', cv=5)
cal_clf.fit(train,target) 

In [ ]:
test_pred = cal_clf.predict_proba(test)

In [ ]:
pd.DataFrame({"ID": id_test, "PredictedProb": test_pred[:,1]}).to_csv('extra_trees_polyCorrParams_calib.csv',index=False)

In [ ]:
test_pred

In [ ]:
#CV Test for xgboost
# XGBoost params:
xgtrain = xgb.DMatrix(train.values, target)
xgtest = xgb.DMatrix(test.values)

In [ ]:
xgboost_params = { 
   "objective": "binary:logistic",
   "booster": "gbtree",
   "eval_metric": "auc",
   "eta": 0.06, # 0.06, #0.01,
   #"min_child_weight": 240,
   "subsample": 0.75,
   "colsample_bytree": 0.68,
   "max_depth": 7
}

In [ ]:
xgb.cv(xgboost_params,xgtrain,num_boost_round=2, nfold=5)

In [ ]:
xgb.cv(xgboost_params,xgtrain,num_boost_round=2, nfold=5)